In [9]:
import sys
import os

# Add the directory containing 'dataloader_prev.py' to the system path
sys.path.append(os.path.abspath('/projectnb/textconv/dgwave/AudioSCC/data/'))
sys.path.append(os.path.abspath('/projectnb/textconv/dgwave/AudioSCC/losses/'))
sys.path.append(os.path.abspath('/projectnb/textconv/dgwave/AudioSCC/models/'))

In [10]:
from dataloader_prev import WaveformDatasetPreload
from sampler_prev import RandomConsecutiveSampler
from losses import ConsecutiveDifferenceHigherOrderLossBatch, ConsecutiveDifferenceHigherOrderLoss
from swissarmy_prev import SeqModel


import numpy as np
import torch
from tqdm import tqdm 
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

In [11]:
# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the first available GPU
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU found, using CPU.")

Using GPU: NVIDIA A40


In [12]:
def generate_sine_tensor(num_bits, length):
    # Create an array of integers from 0 to length - 1
    t = np.arange(length)
    # Generate the sine waves for each bit
    sine_tensor = np.zeros((length, num_bits))  # Initialize the tensor
    
    for i in range(num_bits):
        frequency = (np.pi / (2 ** i))  # Calculate frequency based on the number of bits
        sine_tensor[:, i] = np.cos(frequency * (t))  # Fill the tensor with sine values

    return sine_tensor

In [20]:
!pwd

/projectnb/textconv/dgwave/AudioSCC/notebooks


In [32]:
num_bits = 21
max_len = 100_000
seq_bits = 4
seq_max_len = 2
pred_prev = 5
directory = "/projectnb/textconv/dgwave/AudioSCC//data/digits_two/"
terminal_pad = 11
seq_vocab_len = 10

# Sampler setup as before
batch_size = 2
consecutive_size = 10

In [33]:
t_input = torch.tensor(generate_sine_tensor(num_bits,max_len)).float()
print(t_input.shape)

seq_t = torch.tensor(generate_sine_tensor(seq_bits,seq_max_len)).float()
print(seq_t.shape)

torch.Size([100000, 21])
torch.Size([2, 4])


In [34]:
dataset = WaveformDatasetPreload(directory, t_input, max_len, terminal_pad, seq_vocab_len, seq_max_len, seq_t,pred_prev)

 


sampler = RandomConsecutiveSampler(dataset, batch_size, consecutive_size)

# DataLoader
dataloader = DataLoader(dataset, batch_sampler=sampler, num_workers=8)

In [24]:
dataset.wav_data_list[0].shape

torch.Size([1, 27424])

In [25]:
iteration = 0

for batch in dataloader:
    wav_data, t_step, target, file_idx, seq_inputs, prev_target = batch #right now this wraps arround, just fyi.  not sure its a bad thing.
    if prev_target[0][0] == 0:
        print("Waveform data:", wav_data)
        #print("Time step:", t_step.shape)
        #print("Target tensor:", target.shape)
        #print("File index:", file_idx.shape)
        #print("File index:", seq_inputs.shape)
        print("Previous Predictions:", prev_target)
        #print(wav_data)
        #print(prev_target)
    iteration = iteration + 1
    if iteration > 1000:
        break
    

Waveform data: tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 6.1035e-05],
        [ 0.0000e+00],
        [-1.0675e-01],
        [-1.0434e-01],
        [-9.9335e-02],
        [-9.5428e-02],
        [-9.2712e-02],
        [-9.1705e-02],
        [-9.0454e-02],
        [-9.0332e-02],
        [-9.4177e-02],
        [-9.1156e-02]])
Previous Predictions: tensor([[ 0.0000e+00, -6.1035e-05],
        [-6.1035e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  6.1035e-05],
        [-1.0611e-01, -1.0452e-01],
        [-1.0452e-01, -1.0675e-01],
        [-1.0675e-01, -1.0434e-01],
        [-1.0434e-01, -9.9335e-02]

Waveform data: tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.1697],
        [0.1721],
        [0.1650],
        [0.1565],
        [0.1475],
        [0.1358],
        [0.1198],
        [0.1079],
        [0.0939],
        [0.0750]])
Previous Predictions: tensor([[0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.1584, 0.1637],
        [0.1637, 0.1697],
        [0.1697, 0.1721],
        [0.1721, 0.1650],
        [0.1650, 0.1565],
        [0.1565, 0.1475],
        [0.1475, 0.1358],
        [0.1358, 0.1198],
        [0.1198, 0.1079],
        [0.1079, 0.0939]])
Waveform data: tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00

Waveform data: tensor([[ 6.1035e-05],
        [ 6.1035e-05],
        [ 3.0518e-05],
        [-3.0518e-05],
        [-3.0518e-05],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 6.1035e-05],
        [ 0.0000e+00],
        [ 3.0518e-05],
        [ 3.0518e-05]])
Previous Predictions: tensor([[ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  6.1035e-05],
        [ 6.1035e-05,  6.1035e-05],
        [ 6.1035e-05,  3.0518e-05],
        [ 3.0518e-05, -3.0518e-05],
        [-3.0518e-05, -3.0518e-05],
        [-3.0518e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00]

Waveform data: tensor([[ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 6.1035e-05],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 0.0000e+00],
        [ 2.4719e-03],
        [-7.9346e-03],
        [-2.8992e-03],
        [ 1.1292e-03],
        [-3.0518e-03],
        [ 1.4038e-03],
        [ 4.0894e-03],
        [-6.8970e-03],
        [ 1.1292e-03],
        [-1.0376e-03]])
Previous Predictions: tensor([[ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  6.1035e-05],
        [ 6.1035e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [-4.6387e-03, -1.9226e-03],
        [-1.9226e-03,  2.4719e-03],
        [ 2.4719e-03, -7.9346e-03],
        [-7.9346e-03, -2.8992e-03]

Waveform data: tensor([[0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [6.1035e-05],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00]])
Previous Predictions: tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 6.1035e-05],
        [6.1035e-05, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0

Waveform data: tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0019],
        [ 0.0022],
        [ 0.0035],
        [ 0.0036],
        [-0.0019],
        [ 0.0051],
        [ 0.0008],
        [ 0.0009],
        [ 0.0030],
        [ 0.0020]])
Previous Predictions: tensor([[ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0069,  0.0036],
        [ 0.0036,  0.0019],
        [ 0.0019,  0.0022],
        [ 0.0022,  0.0035],
        [ 0.0035,  0.0036],
        [ 0.0036, -0.0019],
        [-0.0019,  0.0051],
        [ 0.0051,  0.0008],
        [ 0.0008,  0.0009],
        [ 0.0009,  0.0030]])
Waveform data: tensor

In [35]:

config = {
    't_seq_bits': seq_bits,  # Example value for the input bit size
    't_seq_len': seq_max_len,    # Example value for the sequence length
    't_bits': num_bits,      # Example value for the bits used in the decoder

    'encoder': {
        't_layer_dim': 0,               # Example hidden layer dimension for encoder
        't_num_layers': 0,                # Example number of layers in the encoder's initial layer
        'fc_layers': 4,                   # Example number of fully connected layers in the encoder
        'encoder_layers': 2,              # Example number of encoder layers
        'one_hot_vocab_len': 10,          # Vocabulary size for one-hot encoding
        'one_hot_embedding_dim': 20       # Embedding dimension for one-hot encoding
    },

    'decoder': {
        't_layer_dim': 0,                # Example hidden layer dimension for decoder
        't_num_layers': 0,                # Example number of layers in the decoder's initial layer
        'fc_layers': 4,                   # Example number of fully connected layers in the decoder
        'decoder_layers': 3,                # Example number of decoder layers
        'pred_prev_dim': pred_prev
    },

    'output': {
        'mse_output_layers': 2,           # Number of layers in the MSE output head
        'mse_dim': 64,                     # Hidden dimension for the MSE output head
        'bce_output_layers': 2,            # Number of layers in the BCE output head
        'bce_dim': 64                      # Hidden dimension for the BCE output head
    }
}


In [36]:
model = SeqModel(config)

In [37]:
model.to(device)

SeqModel(
  (encoder): SeqEncoder(
    (initial_layer): SwissArmyLayer(
      (t_layers): ModuleList()
      (embedding): Embedding(11, 20, padding_idx=10)
      (layers): ModuleList(
        (0): Linear(in_features=24, out_features=24, bias=True)
        (1): Linear(in_features=24, out_features=24, bias=True)
        (2): Linear(in_features=24, out_features=24, bias=True)
        (3): Linear(in_features=24, out_features=24, bias=True)
      )
    )
    (encoder_layers): ModuleList(
      (0): SwissArmyLayer(
        (t_layers): ModuleList()
        (embedding): Embedding(11, 20, padding_idx=10)
        (layers): ModuleList(
          (0): Linear(in_features=48, out_features=48, bias=True)
          (1): Linear(in_features=48, out_features=48, bias=True)
          (2): Linear(in_features=48, out_features=48, bias=True)
          (3): Linear(in_features=48, out_features=48, bias=True)
        )
      )
      (1): SwissArmyLayer(
        (t_layers): ModuleList()
        (embedding): Embe

In [38]:
bce_output, mse_output = model(seq_inputs,file_idx,t_step,prev_target)
print(bce_output.shape, mse_output.shape)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x95 and 98x98)

In [39]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
prev_target_list = []
mse_loss_fn = nn.MSELoss()
bce_loss_fn = nn.BCELoss()
cdifb_loss = ConsecutiveDifferenceHigherOrderLossBatch(consecutive_size,order=3)
cdif_loss = ConsecutiveDifferenceHigherOrderLoss(consecutive_size,order=3)

num_epochs = 1

# Training loop
for epoch in range(num_epochs):
    model.train()
    
    #for batch in dataloader:
    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        wav_data, t_step, target, file_idx, seq_inputs, prev_target = batch
        
        wav_data = wav_data.to(device)
        t_step = t_step.to(device)
        target = target.to(device)
        file_idx = file_idx.to(device)
        seq_inputs = seq_inputs.to(device)
        prev_target = prev_target.to(device)
        
        
        bce_output, mse_output = model(seq_inputs,file_idx,t_step,prev_target)
        # Compute losses
        mse_loss = mse_loss_fn(mse_output*target, wav_data)  # Assuming the target is for MSE # is this accomplishing what i want?
        bce_loss = bce_loss_fn(bce_output, target)  # Assuming the target is for BCE
        cdif = cdif_loss(mse_output*target, wav_data)
        #bc = bc_loss(outputs, targets)
        cdif_b = cdifb_loss(mse_output*target, wav_data)
        
        
        # Combine losses (you can weight them if needed)
        total_loss = mse_loss + 0.2*bce_loss + 1.9*cdif  + 0.5*cdif_b
        
        # Backpropagation and optimization
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
    # Print progress for each epoch
    print(f"Epoch {epoch+1}/{num_epochs} MSE: {mse_loss.item():.6f} BCE: {bce_loss.item():.6f} CDIF: {cdif.item():.6f}  Total Loss: {total_loss.item():.8f}")
    torch.save(model, "new_model.pth")
print("all done sweetheart <3")

Epoch 1/1:  10%|▉         | 55062/562212 [09:03<1:23:25, 101.32it/s]


KeyboardInterrupt: 

In [ ]:
prev_target_list[-1].shape